# Charge distributions in copper nanostructures

⚠️ Before you embark on this adventure, make sure you have worked your way through the preceding notebook `1_slice_of_numpy.ipynb` to learn the basics of NumPy.

This notebook contains a series of problems to be solved by computing the charge distribution of a copper nanostructure.
All required nanostructures can be loaded from `.txt` files, except for one which you are expected to create with NumPy code.

In [ ]:
# Add additional imports here.
import urllib

import matplotlib.pyplot as plt
import numpy as np

## Configuration of your Notebook environment

### Jupyter Lab and VSCode

You will need to install the following packages using [pip](https://pip.pypa.io/en/stable/installation/): [NumPy](https://numpy.org/), [SciPy](https://scipy.org/), [matplotlib](https://matplotlib.org/) and [ipympl](https://matplotlib.org/ipympl/).
We assume hat you have already done this before starting Jupyter Lab by running the command `pip install numpy scipy matplotlib ipympl`.

### Google Colab

Google Colab comes with NumPy, SciPy and matplotlib packages preinstalled.
However, it lacks support for interactive plots by default.
This can be fixed with the code cell below.
While it technically works with this fix,
interactive plots on Google Colab still tend to be slow.

### Configuration

The following code cell detects whether it is running on Google Colab or not, and executes the appropriate commands to configure interactive plots.

In [ ]:
try:
    from google.colab import output

    output.enable_custom_widget_manager()
    %pip install -q ipympl
except ImportError:
    pass
%matplotlib widget

# Download TXT files

You may already have the TXT files, in which case you can skip the download.
For those using Google Colab, this download is definitely required.

In [ ]:
def download_txt():
    filenames = ["crystal.txt", "plates.txt"]
    for filename in filenames:
        print(f"Downloading {filename}")
        url = f"https://raw.githubusercontent.com/molmod/chargedist/refs/heads/main/{filename}"
        urllib.request.urlretrieve(url, filename)


download_txt()

## Visualization of the charge distribution of a nanostructure

This section introduces a convenient visualization of a charged nanostructure.
Atoms are shown as circles and their color represents their charge.
The first cell defines the function and the second cell shows how to use it.

In [ ]:
def plot_nano(label, atcoords, charges, size=100):
    # Matplotlib 3D scatter plot
    natom = atcoords.shape[0]
    num = f"label_{natom}"
    plt.close(num)
    fig = plt.figure(figsize=(6, 5), num=num)
    ax = fig.add_subplot(projection="3d")
    scale = abs(charges).max()
    sc = ax.scatter(
        atcoords[:, 0],
        atcoords[:, 1],
        atcoords[:, 2],
        c=charges,
        s=size,
        ec="k",
        cmap="coolwarm_r",
        depthshade=False,
        vmin=-scale,
        vmax=scale,
    )
    fig.colorbar(sc, label="Atomic charge [e]")
    qtot = charges.sum()
    ax.set_title(f"Cu$_{{{natom}}}$ nanostructure, $q_\\mathrm{{tot}}$ = {qtot:.2f} e")
    ax.set_xlabel("x [$a_0$]")
    ax.set_ylabel("y [$a_0$]")
    ax.set_zlabel("z [$a_0$]")
    ax.dist = 10
    ax.set_box_aspect(
        aspect=(np.ptp(atcoords[:, 0]), np.ptp(atcoords[:, 1]), np.ptp(atcoords[:, 2]))
    )

The following is a small demo of the visualization function.
It loads the nanocrystal from disk and visualizes it, using the x-coordinate as a surrogate for the partial charges.

If all goes well, you should be able to rotate the 3D view by dragging with the mouse.

In [ ]:
atcoords = np.loadtxt("crystal.txt")
plot_nano("demo", atcoords, charges=atcoords[:, 0])

## Challenge 1: the charge distribution of a nanocrystal

Now it is up to you: you should be able to combine all the information from the theory slides and the previous notebook to construct the correct linear system of equations to calculate the charges.
Do this calculation for a crystal with net charge of $+10\,\mathrm{e}$.
Once you have solved the equations, you can visualize the atomic charges using the `plot_nano` function.
You should get a result similar to the "goals" slide from the theory part.

You can check your answer by computing the potential energy of the optimal charge distribution. This should be $U_\text{mol}\approx-0.43286\,\mathrm{E_h}$.

While solving and visualizing the charges, try to answer the following questions:

- **Q1.** What features of the charge distribution can you explain with what you have learned from the electrostatics chapters?
- **Q2.** Construct a boolean index array to select atoms with a negative $x$-coordinate.
  Use this boolean array to visualize only one half of the crystal, so that you can look inside.
  Can you explain the internal charge distribution?
- **Q3.** Compute the energy of the charged particle and compare it with the energy of a charged metal sphere of approximately the same radius.
  Do you get consistent results?

In [ ]:
# Write your code here

## Challenge 2: A nanoscale capacitor consisting of two crystaline slabs

The file `plates.txt` contains two parallel copper plates that can be used to model a nanoscale capacitor.
Each plate consists of an equal number copper atoms.

Load this file and compute a charge distribution when one plate is constrained to have a charge of $+10\,e$ and the other plate is constrained to have a charge of $-10\,e$.

As you solve and visualize the charges, try to answer the following questions:

- **Q4.** How can you derive the capacitance? If implemented correctly, you should find a capacitance $C\approx 6.2533\,\mathrm{E_h e^{-2}}$.
- **Q5.** Try to estimate the capacitance by approximating this capacitor as an ideal flat plate capacitor.
  Do you get consistent results?
- **Q6 (difficult).** What is the physical meaning of the Lagrange multipliers in this case?

In [ ]:
# Write your code here

## Optional extra challenges

For those with good programming skills, here are some additional challenges to keep you entertained. To get acceptable performance, you have to be careful how you vectorize the calculations with NumPy.

- Visualize the electric field in the $(x, y)$-plane of the capacitor with a [stream plot](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.streamplot.html).
  Do you see a region where the field is homogeneous?

- Visualize the isopotential surfaces for the capacitor model.
  You can do this using the [Marching Cubes](https://en.wikipedia.org/wiki/Marching_cubes) algorithm as implemented in [scikit-image](https://scikit-image.org/docs/stable/auto_examples/edges/plot_marching_cubes.html#marching-cubes).
  Do you see flat surfaces between the plates?
  